In [1]:
%run ~/relmapping/annot/notebooks/__init__.ipynb

/mnt/home3/jj374/anaconda36/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


os.getcwd(): /mnt/b2/scratch/ahringer/jj374/lab/relmapping


In [2]:
fp = 'annot/S2_regulatory_annotation/_lcap_jump/lcap_counts.tsv'
#fp = 'annot/S2_regulatory_annotation/_lcap_jump/lcap_counts_incr.tsv'
df = pd.read_csv(fp, sep='\t')
df.head(5)

,chrom,start,end,atac_mode,lcap_wt_emb_rep1_fwd_rcount,lcap_wt_emb_rep2_fwd_rcount,lcap_wt_emb_rep1_fwd_fcount,lcap_wt_emb_rep2_fwd_fcount,lcap_wt_emb_rep1_rev_fcount,lcap_wt_emb_rep2_rev_fcount,...,lcap_glp1_d10_rep1_rev_rcount,lcap_glp1_d10_rep2_rev_rcount,lcap_glp1_d14_rep1_fwd_rcount,lcap_glp1_d14_rep2_fwd_rcount,lcap_glp1_d14_rep1_fwd_fcount,lcap_glp1_d14_rep2_fwd_fcount,lcap_glp1_d14_rep1_rev_fcount,lcap_glp1_d14_rep2_rev_fcount,lcap_glp1_d14_rep1_rev_rcount,lcap_glp1_d14_rep2_rev_rcount
0,chrI,1870,2092,1981,0,0,0,1,0,0,...,38,9,1,1,1,0,8,0,12,4
1,chrI,3809,3994,3901,0,0,1,0,1,0,...,81,172,1,5,5,3,0,0,129,61
2,chrI,4302,4410,4356,1,1,0,1,2,3,...,9,3,7,8,8,7,3,1,7,6
3,chrI,11245,11465,11355,3,1,7,5,9,12,...,2,4,1,0,51,47,1,0,10,2
4,chrI,13087,13207,13147,10,11,9,11,29,26,...,10,14,101,142,114,133,19,9,9,17


In [3]:
def deseq2x2(df_counts):
    fh_inp, fp_inp = tempfile.mkstemp(dir=os.getcwd(), prefix='deseq2x2_inp_')
    fh_out, fp_out = tempfile.mkstemp(dir=os.getcwd(), prefix='deseq2x2_out_')
    df_counts.to_csv(fp_inp, sep='\t')
    !cat {fp_inp} | scripts/deseq2x2_greater.R > {fp_out}
    #!wc -l {fp_inp}
    #!wc -l {fp_out}
    #!tail -n 20 {fp_out}
    df_out = pd.read_csv(fp_out, sep='\s+')
    !rm {fp_inp}
    !rm {fp_out}
    return df_out

l_df_fwd = []
l_df_rev = []
for stage in itertools.islice(config['lcap_geo_by_stage'], None):
    print(stage)
    df_fwd = deseq2x2(df[[
        'lcap_%(stage)s_rep1_fwd_rcount' % locals(),
        'lcap_%(stage)s_rep2_fwd_rcount' % locals(),
        'lcap_%(stage)s_rep1_fwd_fcount' % locals(),
        'lcap_%(stage)s_rep2_fwd_fcount' % locals(),]]#.head(100)
    )
    df_rev = deseq2x2(df[[
        'lcap_%(stage)s_rep1_rev_fcount' % locals(),
        'lcap_%(stage)s_rep2_rev_fcount' % locals(),
        'lcap_%(stage)s_rep1_rev_rcount' % locals(),
        'lcap_%(stage)s_rep2_rev_rcount' % locals(),]]#.head(100)
    )
    df_fwd.columns = ['lcap_%s_fwd_%s' % (stage, column) for column in df_fwd.columns]
    df_rev.columns = ['lcap_%s_rev_%s' % (stage, column) for column in df_rev.columns]
    l_df_fwd.append(df_fwd)
    l_df_rev.append(df_rev)

df_deseq = pd.concat(l_df_fwd + l_df_rev, axis=1)

wt_emb
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
wt_l1
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
wt_l2
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
wt_l3
estimating size

In [4]:
df_deseq.to_csv('annot/S2_regulatory_annotation/_lcap_jump/lcap_deseq.tsv', sep='\t', index=False)